In [1]:
# =========================
# Library
# =========================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import os
import gc
import random
from glob import glob
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold
import warnings
import seaborn as sns
import pickle
import json
import re
import time
import sys
from requests import get
import multiprocessing
import joblib
import Levenshtein
import difflib
from contextlib import contextmanager
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import log_loss
import logging
warnings.filterwarnings('ignore')
import sys
sys.path.append("../src/")
from logger import setup_logger, LOGGER
from trainer import train_lgbm
from util_tool import reduce_mem_usage
pd.set_option('display.max_columns', 300)

In [2]:
# =========================
# Constant
# =========================
TRAIN_PATH = "../data/train.csv"
TARGET = "point_of_interest"

In [3]:
# =========================
# Settings
# =========================
exp = "063"
val_fold = 1

fe_list = ["../output/fe/fe066.pkl",
           "../output/fe/fe067.pkl",
           "../output/fe/fe068.pkl",
           "../output/fe/fe069.pkl"]

In [5]:
# ==========================
# Main
# ==========================
train = pd.read_pickle(fe_list[0]) 
for i in range(1,len(fe_list)):
    _train = pd.read_pickle(fe_list[i])
    train = pd.concat([train,_train],axis=1)
    del _train
    gc.collect()

In [6]:
train_tr = train[train["set"] != val_fold].reset_index(drop=True)
val_tr = train[train["set"] == val_fold].reset_index(drop=True)
del train
gc.collect()

23

In [7]:
y_pred_valid = np.load(f"../output/exp/ex{exp}/oof.npy")

In [8]:
val_tr["pred"] = y_pred_valid

In [9]:
# 0.01以上を残す
val_tr = val_tr[val_tr["pred"] >= 0.01].reset_index(drop=True)

In [10]:
val_tr.to_csv("../output/exp/ex063/ex063_fe.csv",index=False)